In [ ]:
import h5py
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import CubicSpline
from TurbSound import TurbSound

In [ ]:
#########
#### The output dimensions (N_samples, 16, 275) correspond to the number of oberservers.
#### Observers are stored as model attributes in both polar (theta, r) and Cartesian (x, y) 
#### coordinates.
#########
bounds = [[4., 25.],     # Wind Speed w/ extra weighting [4, 15]
          [0.15, 0.4],   # Mean T.I. w/ slope of 0.2/140
          [-10., 35.],   # Air Temperature
          [10., 95.],    # Relative Humidty
          [80., 105.],   # Air Pressure
          [0., 0.5]]     # Ground Factor
bounds = np.array(bounds)

# Initialize model - model_path points to the saved model weights
model = TurbSound(model_path='model/model.h5')

print('Polar Coordinates of Observers')
print(model.theta.shape, model.r.shape)

print('Cartesian Coordinates of Observers')
print(model.x.shape, model.y.shape)


# Plot the observers colored by their angular location
plt.figure(figsize=(6, 4))
plt.scatter(model.x, model.y, c=model.theta, s=5)
plt.colorbar(label='Angular Location (deg.)')
plt.gca().set_aspect(1.)
plt.title('Observers')
plt.show()

In [ ]:
#########
#### This repository includes example data to use for demonstration. This includes one year of
#### hourly weather data taken from five different locations. The data was collected from the
#### WIND Toolkit (https://www.nrel.gov/grid/wind-toolkit.html).
#########

# Extract data for 'example0X' (X = 1, 2, 3, 4, 5)
with h5py.File('example_data.h5', 'r') as hf:
    k = 'example01'

    # All atmospheric values are (8760, ) except the ground factor, which is a scalar
    ws = hf[k]['wind_speed'][()]
    wd = hf[k]['wind_direction'][()]
    ti = hf[k]['turb_intensity'][()]
    at = hf[k]['air_temperature'][()]
    rh = hf[k]['rel_humidity'][()]
    ap = hf[k]['air_pressure'][()]
    gf = hf[k]['ground_factor'][()]

# The data is packaged as a dictionary and an array for demonstration of the model
x_dict  =  {'wind_speed': ws, 'wind_direction': wd, 'turb_intensity': ti, 
            'air_temperature': at, 'rel_humidity': rh, 'air_pressure': ap, 'ground_factor': gf}
x_array = np.concatenate((ws.reshape((1, -1)), wd.reshape((1, -1)), ti.reshape((1, -1)),
                          at.reshape((1, -1)), rh.reshape((1, -1)), ap.reshape((1, -1)),
                          gf*np.ones((1, ws.size))), axis=0)

months = ['J', 'F', 'M', 'A', 'M', 'J', 
          'J', 'A', 'S', 'O', 'N', 'D']

plt.figure(figsize=(12, 5))

plt.subplot(2, 3, 1)
plt.plot(ws, c='b', lw=0.25)
plt.xticks(np.linspace(365, 8395, 12), labels=months)
plt.xlabel('Month')
plt.ylabel('Wind Speed (m/s)')
plt.subplot(2, 3, 2)
plt.plot(wd, c='b', lw=0.25)
plt.xticks(np.linspace(365, 8395, 12), labels=months)
plt.xlabel('Month')
plt.ylabel('Wind Direction (deg.)')
plt.subplot(2, 3, 3)
plt.plot(ti, c='b', lw=0.25)
plt.xticks(np.linspace(365, 8395, 12), labels=months)
plt.xlabel('Month')
plt.ylabel('Turbulence Intensity')
plt.subplot(2, 3, 4)
plt.plot(at, c='b', lw=0.25)
plt.xticks(np.linspace(365, 8395, 12), labels=months)
plt.xlabel('Month')
plt.ylabel('Air Temperature (deg. C)')
plt.subplot(2, 3, 5)
plt.plot(rh, c='b', lw=0.25)
plt.xticks(np.linspace(365, 8395, 12), labels=months)
plt.xlabel('Month')
plt.ylabel('Relative Humidity (%)')
plt.subplot(2, 3, 6)
plt.plot(ap, c='b', lw=0.25)
plt.xticks(np.linspace(365, 8395, 12), labels=months)
plt.xlabel('Month')
plt.ylabel('Air Pressure (kPa)')

plt.suptitle('Example {} --- Ground Factor = {:.2f}'.format(k[-2:], gf))
plt.tight_layout()
plt.show()

In [ ]:
#########
#### The base function of the surrogate model is the model.compute_sound() function, which takes in
#### a single or a series of atmospheric conditions and outputs the associated sound level at each
#### observer location.
#########

# Extract a single time step and compute the sound level
idx = np.random.randint(x_array.shape[1])
x_single = x_array[:, idx]
f_single = model.compute_sound(x=x_single)
print('Input size: {} --- Output size: {}'.format(x_single.shape, f_single.shape))

# Run the model on the full year of data
f_array = model.compute_sound(x=x_array)
print('Input size: {} --- Output size: {}'.format(x_array.shape, f_array.shape))

# Run the model using the dictionary input
f_dict = model.compute_sound(x=x_dict)
print('Input size: {} --- '.format([(k, x_dict[k].shape) for k in x_dict]), end='')
print('Output size: {}'.format(x_array.shape, f_array.shape))

In [ ]:
#########
#### Predicted sound levels can be plotted using the model observer locations (x,y) or (r,theta)
#########

# Extract a single time step and compute the sound level
f = model.compute_sound(x=x_array[:, np.random.randint(x_array.shape[1])])[0]


plt.figure(figsize=(10, 4))

plt.subplot(121)
plt.scatter(model.x, model.y, c=f, s=5)
plt.colorbar()
plt.gca().set_aspect(1.)

# For contour plot, complete the circle
x = np.concatenate((model.x, model.x[:1, :]), axis=0)
y = np.concatenate((model.y, model.y[:1, :]), axis=0)
f = np.concatenate((f, f[:1, :]), axis=0)

plt.subplot(122)
plt.contourf(x, y, f)
plt.colorbar()
plt.gca().set_aspect(1.)

plt.show()

In [ ]:
#########
#### Given a timeseries of atmospheric data, we may not want the full block of sound levels.
#### Instead, we want to compute statistical metrics of the sound level. We can do this 
#### using the function model.sound_metrics(). 
#########

# A batch_size parameter is provided to reduce memory costs of processing large timeseries.
# Larger batch_size can accelerate the speed but will require more memory.
# Set batch_size = -1 to automatically run the entire timeseries as a single batch.
batch_size = 1000

# Flag to check the bounds of the input values. Will send a warning if any inputs are outside
# of the expected bounds.
check_bounds = False

##### NOTE: that we can run a single metric_list at a time. For example:
Lmax = model.sound_metrics(metric_list='Lmax', x=x_dict, batch_size=batch_size, check_bounds=check_bounds)
Leq = model.sound_metrics(metric_list='Leq', x=x_dict, batch_size=batch_size, check_bounds=check_bounds) 
L50 = model.sound_metrics(metric_list='L50', x=x_dict, batch_size=batch_size, check_bounds=check_bounds) 
##### Or we can run multiple metric_list's at once (This is more efficient)
L = model.sound_metrics(metric_list=['Lmax', 'Leq', 'L50'], x=x_dict, batch_size=batch_size, check_bounds=check_bounds)
Lmax, Leq, L50 = L[0], L[1], L[2]

# Regardless of metric, output is the statistical sound level for each observer
print('Output shape of various stat metrics:')
print(Lmax.shape, Leq.shape, L50.shape)
print('')

# Processing for contour plot visualization
x = np.concatenate((model.x, model.x[:1, :]), axis=0)
y = np.concatenate((model.y, model.y[:1, :]), axis=0)
Lmax = np.concatenate((Lmax, Lmax[:1, :]), axis=0)
Leq = np.concatenate((Leq, Leq[:1, :]), axis=0)
L50 = np.concatenate((L50, L50[:1, :]), axis=0)

vmin, vmax = 0, np.max(np.concatenate((Lmax, Leq, L50), axis=0))

plt.figure(figsize=(14, 3))

plt.subplot(131)
plt.contourf(x, y, Lmax, levels=np.linspace(vmin, vmax, 15))
plt.colorbar()
plt.title('Lmax')
plt.gca().set_aspect(1.)

plt.subplot(132)
plt.contourf(x, y, Leq, levels=np.linspace(vmin, vmax, 15))
plt.colorbar()
plt.title('Leq')
plt.gca().set_aspect(1.)

plt.subplot(133)
plt.contourf(x, y, L50, levels=np.linspace(vmin, vmax, 15))
plt.colorbar()
plt.title('L50')
plt.gca().set_aspect(1.)

plt.show()

In [ ]:
#########
#### Given sound levels at the various observers (this may be for a single scenario but
#### is more likely to be metric-based), we can compute setback distances for a 
#### specified sound_threshold level using model.sound_setback().
#########
#### model.sound_setback() has the following inputs
####      L                 - (NumPy array) Sound level output from either model.compute_sound()
####                                        or model.sound_metrics
####      sound_threshold   - (float) Threshold sound level to compute setback distance
#### model.sound_setback() outputs an array of 8 sectback distances, each corresponding to
#### to an angular spoke of our observers. These angles can be obtained from model.theta[:, 0]
#########

# Set the list of metrics for which to compute statistical sound levels
metric_list = ['Lmax', 'Leq', 'L10', 'L25', 'L50', 'L75', 'L90']
# Set the sound level threshold to determine the setback distance
sound_threshold = 45.

# Compute the sound metrics
L = model.sound_metrics(metric_list=metric_list, x=x_dict, batch_size=1000, check_bounds=False)

plt.figure(figsize=(6, 6))
for i, L_i in enumerate(L):
    setback = model.sound_setback(L_i, sound_threshold)

    # Refining the setback curve to make a fun plot
    theta_coarse = np.concatenate((model.theta[:, 0], [360]), axis=0)
    theta = np.linspace(0., 360, 100)
    setback = np.concatenate((setback, setback[:1]), axis=0)

    cs = CubicSpline(theta_coarse, setback, bc_type='periodic')
    setback = cs(theta)

    plt.plot(setback*np.cos((np.pi/180.)*theta), setback*np.sin((np.pi/180.)*theta), label=metric_list[i])
plt.legend(bbox_to_anchor=(1.05, 1))
plt.gca().set_aspect(1.)
plt.show()

In [ ]:
#########
#### The functions model.sound_metrics() and model.sound_setback() can be collapsed
#### to the 1D case by adding the flag worst_case=True to each function call.
####
#### In this case, the sound level output is 1D along the range of observer radii,
#### which can be obtained from model.r[0, :]. Similarly, the setback distance will
#### be scalar-valued rather than an array of distances at different angles.
#########

sound_threshold = 45.

L = model.sound_metrics(metric_list=['Lmax', 'L50', 'Leq'], x=x_dict, batch_size=1000, check_bounds=False)
Lmax, L50, Leq = L[0], L[1], L[2]

setback_Lmax = model.sound_setback(Lmax, sound_threshold, worst_case=True)
setback_L50 = model.sound_setback(L50, sound_threshold, worst_case=True)
setback_Leq = model.sound_setback(Leq, sound_threshold, worst_case=True)

ymin, ymax = 0, np.max(np.concatenate((Lmax, Leq, L50), axis=0)) + 5.

plt.figure(figsize=(14, 3))

plt.subplot(131)
plt.plot(model.r[0, :], np.max(Lmax, axis=0), 'b')
plt.plot([0, setback_Lmax], [sound_threshold, sound_threshold], '--r')
plt.plot([setback_Lmax, setback_Lmax], [0, sound_threshold], '--r')
plt.xlim(0, np.max(model.r))
plt.ylim(ymin, ymax)
plt.title('Lmax')

plt.subplot(132)
plt.plot(model.r[0, :], np.max(L50, axis=0), 'b')
plt.plot([0, setback_L50], [sound_threshold, sound_threshold], '--r')
plt.plot([setback_L50, setback_L50], [0, sound_threshold], '--r')
plt.xlim(0, np.max(model.r))
plt.ylim(ymin, ymax)
plt.title('L50')

plt.subplot(133)
plt.plot(model.r[0, :], np.max(Leq, axis=0), 'b')
plt.plot([0, setback_Leq], [sound_threshold, sound_threshold], '--r')
plt.plot([setback_Leq, setback_Leq], [0, sound_threshold], '--r')
plt.xlim(0, np.max(model.r))
plt.ylim(ymin, ymax)
plt.title('Leq')

plt.show()